In [16]:
import pandas as pd
import re

In [17]:
import devanalyst.simulation.statics as S_

<h1>Data Structures</h1>

In [2]:
class UserStory:
# Represents a unit of *planned* work, intended to be completed in one sprint by 1 developer based on a specification from
# a product manager
#
# -userStoryId: a string unique identifier of this user story. Example: "UserStory #65"
# -originalEstimate: a float, representing the number of man-days user story was supposed to take. Estimated by developer 
# based on the spec.
# -developer: name of the developer responsible for this user story.
# -productManager: name of product manager who wrote the original spec for this user story

    def __init__(self, userStoryId, originalEstimate, developer, productManager):
        self.userStoryId = userStoryId
        self.originalEstimate = originalEstimate
        self.developer = developer
        self.productManager = productManager
    
    # Static method to create a DataFrame with all the user stories.
    # -globalRepo: a GlobalRepo instance with all persistent information pertinent to this release.
    def build_stories_df(globalRepo):
        stories_dict = {}
        cols = ['User Story Id', 'Original Estimate', 'Team Id', 'Developer', 'Product Manager', 'Percent Achieved', \
                'Planned', 'Sprint Planned', 'Sprint Delivered', 'Nb Open Bugs', 'Open Bugs', 'Nb Closed Bugs', 'Closed Bugs']
        for c in cols:
            stories_dict[c] = []

        storiesRepo    = globalRepo.storiesRepo
        teamsRepo      = globalRepo.teamsRepo
        ticketsRepo    = globalRepo.ticketsRepo
        for storyId in storiesRepo.findIds():
            story        = storiesRepo.findStory(storyId)
            uss          = teamsRepo.getUserStoryStatus(storyId)
            open_bugs    = ticketsRepo.getOpenTickets(storyId)
            closed_bugs  = ticketsRepo.getClosedTickets(storyId)


            open_bugs_ids = []
            for bug in open_bugs:
                open_bugs_ids.append(bug.ticketId)

            closed_bugs_ids = []
            for bug in closed_bugs:
                closed_bugs_ids.append(bug.ticketId)


            stories_dict['User Story Id']       .append(story.userStoryId)
            stories_dict['Original Estimate']   .append(story.originalEstimate)
            stories_dict['Team Id']             .append(teamsRepo.getTeamId(storyId))
            stories_dict['Developer']           .append(story.developer)
            stories_dict['Product Manager']     .append(story.productManager)
            stories_dict['Percent Achieved']    .append(uss.percentAchieved)
            stories_dict['Planned']             .append(uss.planned)
            stories_dict['Sprint Planned']      .append(uss.sprintPlanned)
            stories_dict['Sprint Delivered']    .append(uss.sprintDelivered)        
            stories_dict['Nb Open Bugs']        .append(len(open_bugs_ids))   
            stories_dict['Open Bugs']           .append(open_bugs_ids)        
            stories_dict['Nb Closed Bugs']      .append(len(closed_bugs_ids))        
            stories_dict['Closed Bugs']         .append(closed_bugs_ids)        

        stories_df = pd.DataFrame(stories_dict)
        return stories_df

In [19]:
class UserStoriesRepo:
# Represents an in-memory proxy for a repo of user stories.
#
# -stories: a list of UserStory objects.

    def __init__(self, stories):
        self.stories = stories
        
    def findStory(self, storyId):
        for story in self.stories:
            if story.userStoryId == storyId:
                return story
        return None
    
    # Returns a list of all the userStoryIds for all the UserStories in this repo
    def findIds(self):
        ids = []
        for story in self.stories:
            ids.append(story.userStoryId)
        return ids

In [1]:
class Ticket:
# Represents a *non-planned* task needed to complete a user story. Typically these might be: bugs, or the need to rework
# something because the spec was misunderstood by the developer.
#
# -ticketId: string unique identifier for this ticket. Must be of the form "Ticket #n" for some integer n
# -userStoryId: a string unique identifier of the user story for which this is an task. Example: "UserStory #65"
# -estimatedCost: a non-negative integer to convey how difficulty it will be to fix. Recommended values between 1 and 10.
# -sprintReported: an integer corresponding to the number of the sprint in which this ticket was raised.
# -effortToDate: number of man-days spent fixing this ticket so far
# -percentAchieved: percentage of issue that has been fixed

    def __init__(self, ticketId, userStoryId, estimatedCost, sprintReported):
        self.ticketId         = ticketId
        self.userStoryId      = userStoryId
        self.estimatedCost    = estimatedCost
        self.sprintReported   = sprintReported
        self.effortToDate     = 0.0
        self.percentAchieved  = 0.0
        self.sprintFixed      = S_.NOT_SET
        self.fixedBy          = S_.NOT_SET
        
    # Computes how long (in man-days) it should take to complete this ticket.
    def estimateRemainingEffort(self):         
        return self.estimatedCost * (1 - self.percentAchieved)
    
    # Static method to create a DataFrame out of a lis t of Tickets. Useful to visualize things when testing.
    def build_bugs_df(bugs):
        # -bugs: a list of Tickets
        bugs_dict = {}
        bugs_dict['Ticket Id']        = []
        bugs_dict['User Story Id']    = []
        bugs_dict['Estimated Cost']   = []
        bugs_dict['Effort to Date']   = []
        bugs_dict['Percent Achieved'] = []
        bugs_dict['Sprint Reported']  = []
        bugs_dict['Sprint Fixed']     = []
        bugs_dict['Fixed By']         = []

        for bug in bugs:
            bugs_dict['Ticket Id']        .append(bug.ticketId)
            bugs_dict['User Story Id']    .append(bug.userStoryId)
            bugs_dict['Estimated Cost']   .append(bug.estimatedCost)
            bugs_dict['Effort to Date']   .append(bug.effortToDate)
            bugs_dict['Percent Achieved'] .append(bug.percentAchieved)
            bugs_dict['Sprint Reported']  .append(bug.sprintReported)
            bugs_dict['Sprint Fixed']     .append(bug.sprintFixed)
            bugs_dict['Fixed By']         .append(bug.fixedBy)

        bugs_df = pd.DataFrame(bugs_dict)
        return bugs_df

In [21]:
class TicketsRepo:
# Represents an in-memory proxy for a repo of tickets
#
# -tickets: a list of Ticket objects.

    def __init__(self):
        self.tickets = []
        
    def findTicket(self, ticketId):
        for ticket in self.tickets:
            if ticket.ticketId == ticketId:
                return ticket
        return None         

    def addTicket(self, ticket):
        self.tickets.append(ticket)
        
    def createTicket(self, userStoryId, costToFix):
        ticketId = self._nextTicketId()
        ticket = Ticket(ticketId, userStoryId, costToFix)
        self.tickets.append(ticket)
        return ticket
    
    def removeTicket(self, ticketId):
        ticket = self.findTicket(ticketId)
        if (ticket != None):
            self.tickets.remove(ticket)
                
    def updateTicket(self, ticket):
        self.removeTicket(ticket.ticketId)
        self.tickets.append(ticket)
   
    # Returns a list of Ticket instances, all of them 'open' (i.e., percentAchieved < 1.0) and associated to the
    # UserStory referenced by the 'userStoryId' parameter
    def getOpenTickets(self, userStoryId):
        bugs = []
        for t in self.tickets:
            if t.userStoryId == userStoryId and t.percentAchieved < 1.0:
                bugs.append(t)
        return bugs
    
    # Returns a list of Ticket instances, all of them 'closed' (i.e., percentAchieved == 1.0) and associated to the
    # UserStory referenced by the 'userStoryId' parameter
    def getClosedTickets(self, userStoryId):
        bugs = []
        for t in self.tickets:
            assert(t.percentAchieved <= 1.0)
            if t.userStoryId == userStoryId and t.percentAchieved == 1.0:
                bugs.append(t)
        return bugs

    # Returns a valid ticketId string that does not correspond to any Ticket in this repo
    def _nextTicketId(self):
        # ticketIds look like 'Ticket #n' for some integer n. Find the next n that hasn't been used yet
        last_number_used = 0
        pattern = '[0-9]+'
        for t in self.tickets:
            m = re.search(pattern, t.ticketId)
            id_number = int(m[0])
            last_number_used = max(last_number_used, id_number)
        next_number = last_number_used + 1
        nextId = 'Ticket #' + str(next_number)
        assert(self.findTicket(nextId) == None) # nexId should note be in use already
        
        return nextId

In [22]:
class WorkItem:
# Represents an item of work. This class is used to list all the possible things to do in a sprint (such as finishing
# a user story or fixing a ticket).
# 
# -userStoryId: the user story for which this is part of the work
# -taskType: string. Can be one of: S_.UNFINISHED_STORIES (to develop the user story) and S_.BUGS_ON_COMPLETED_STORIES or
# S_.BUGS_ON_UNFINISHED_STORIES (to fix tickets)
# -estimate: a float representing the amount of time the work it was believed to take at the start of the sprint, in man-days.
# -percentAchieved: a float representing the percentage of the work that got finished by the end of the sprint after spending
# the 'estimate' number of man-days.
# -owner: a string representing the person responsible for implementing this user story, as well as fixing anything wrong
# with it.
# -sprintPlanned: a boolean. Records whether this item was chosen for delivery in a sprint.
# -ticketId: identifier of the ticket for work items with a taskType of 'REWORK'. Otherwise it is not set

    def __init__(self, userStoryId, taskType, estimate, percentAchieved, owner, sprintPlanned, ticketId = None):
        self.userStoryId       = userStoryId
        self.taskType          = taskType
        self.ticketId          = ticketId #Only pertinent if the taskType is S_.BUGS_ON_COMPLETED_STORIES or S_.BUGS_ON_UNFINISHED_STORIES
        self.estimate          = estimate
        self.percentAchieved   = percentAchieved
        self.owner = owner
        
        # Set at the end of a sprint where this WorkItem is assigned. If work is completed and no delays exist, 
        # will match the self.estimate when it is set
        self.actual = 0.0
        
        self.sprintPlanned = sprintPlanned # Records in which sprint this user story was first planned to be delivered in

In [23]:
class ReleaseCycleContext:
# Structure used to hold runtime information about a particular point in the release cycle. This contextual information
# may be relied upon by simulation models whose calculations are dependent what has previously happened in the 
# release cycle. 
# This context can also be helpful in debugging by, for example, indicating where in a release cycle (e.g., in which
# sprint) some anomalous behavior is detected.

    def __init__(self, teamId, sprint, sprintDuration):
        self.teamId          = teamId
        self.sprint          = sprint
        self.sprintDuration  = sprintDuration

In [2]:
class WorkAssignments:
# Main data structure class to maintain how work is assigned to developers of a particular Team at a particular point in time, e.g.,
# for a particular sprint. It optionally can maintain work assigned to developers for the *next* sprint, but which
# might be started during the current sprint if developers finish early their current sprint's obligations. 
#
# -context: identifies the ReleaseCycleContext instance pertinent for the work assigned in this 'WorkAssignment' instance
# -globalRepo: instance of GlobalRepo where all persistent information is stored.
# -allocations: a nested dictionary to partition the backlog into indidual buckets based on whether they are planned, 
# in which case for which sprint (CURRENT_SPRINT, NEXT_SPRINT, UNPLANNED) and the developer assigned to, or whether they 
# are unplanned. As an added breakdown, we break by task type (UNFINISHED_STORY, DEV_TIME_BUGS,
# or PRODUCTION_BUGS).
# "Normally", the NEXT_SPRINT bucket would be "empty", since at the start of the sprint we assign only work for the
# CURRENT_SPRINT, and the rest of the backlog should therefore be UNPLANNED. However, since UserStories are not all of the
# same size, it can happen that developers finish their CURRENT_SPRINT early, so rather than keeping idle they shoulds start
# to progress some of the work for the NEXT_SPRINT. That is why that bucket exists, to give them something to do if
# all is delivered for the CURRENT_SPRINT, even if they can only be delivered by the NEXT_SPRINT
#
# Example of the 'allocations' attribute of this class:
#
# {CURRENT_SPRINT: {'Joe Developer':      {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          DEV_TIME_BUGS:               [list of WorkItems ...],
#                                          PRODUCTION_BUGS:             [list of WorkItems ...]},
#                   'Gladys Developer':   {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          DEV_TIME_BUGS:               [list of WorkItems ...],
#                                          PRODUCTION_BUGS:             [list of WorkItems ...]}    },                                                                                                        
#  NEXT_SPRINT:    {'Joe Developer':      {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          DEV_TIME_BUGS:               [list of WorkItems ...],
#                                          PRODUCTION_BUGS:             [list of WorkItems ...]},
#                   'Gladys Developer':   {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          DEV_TIME_BUGS:               [list of WorkItems ...],
#                                          PRODUCTION_BUGS:             [list of WorkItems ...]}    },
#  UNPLANNED:      {OWNER_TBD:            {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          DEV_TIME_BUGS:               [list of WorkItems ...],
#                                          PRODUCTION_BUGS:             [list of WorkItems ...]}    }    }
    
    def __init__(self, context, globalRepo):
        self.context       = context
        self.globalRepo    = globalRepo
        
        # Set the initial value of 'allocations' by looking at all work items in the team's backlog, and then
        # classifying them into several buckets:
        self.allocations   = {S_.CURRENT_SPRINT: {}, S_.NEXT_SPRINT: {}, S_.UNPLANNED: {}}
        team               = self.globalRepo.teamsRepo.findTeam(self.context.teamId)
        for person in team.developers:
            for bucket in [S_.CURRENT_SPRINT, S_.NEXT_SPRINT]:
                self.allocations[bucket][person] = {S_.PRODUCTION_BUGS: [], \
                                                    S_.DEV_TIME_BUGS: [], \
                                                    S_.UNFINISHED_STORIES: []}

        self.allocations[S_.UNPLANNED][S_.OWNER_TBD] = {S_.PRODUCTION_BUGS: [], \
                                                        S_.DEV_TIME_BUGS: [], \
                                                        S_.UNFINISHED_STORIES: []}
        
        work = team.backlog._generateWorkItems(self.context, self.globalRepo)
        for item in work:
            self.assign(item, item.owner, S_.CURRENT_SPRINT)

    def assign(self, item, owner, preferredBucket):
        # Needs to record owner in two places: in the self.allocations, and if the owner is a real person (not the
        # OWNER_TBD global, then that means the UserStoryStatus in question is officially planned, so must
        # mutate the UserStoryStatus to reflect that.
        # Potentially the owner being assigned differs from the original developer associated to the User Story
        # at the start of the release (that association was driven purely from capacity planning, not the more micro
        # sprint planning we are doing now). So if the owner is a real person then need to update the user story as well
        story     = self.globalRepo.storiesRepo.findStory(item.userStoryId)
        team      = self.globalRepo.teamsRepo.findTeam(self.context.teamId)
        uss       = team.backlog.getUserStoryStatus(item.userStoryId)
        
        item.owner = owner # Update who owns this item
        
        old_owner = story.developer
        
        if (owner != S_.OWNER_TBD):
            # We actually are giving this to someone, which means it is a planned sprint deliverable
            story.developer = owner # May change the developer who initially had this in the backlog, still same scrum team
            uss.planned = True
            assigningForTheFirstTime = False
            if (uss.sprintPlanned == S_.NOT_SET): #This is the first sprint for which we are planning to work on this story
                assigningForTheFirstTime = True
                uss.sprintPlanned = self.context.sprint
                item.sprintPlanned = self.context.sprint #Needed as item as the uss's old value for sprintPlanned when item was created

            # Record assignment event in uss if either this is the first time or if we are changing woner
            if (assigningForTheFirstTime or owner != old_owner):
                uss.logAssignmentEvent(self.context.sprint, owner)
            
        bucketToUse = preferredBucket
        if (owner == S_.OWNER_TBD):
            bucketToUse = S_.UNPLANNED
            
        if (item.taskType == S_.PRODUCTION_BUGS):
            self.allocations[bucketToUse][owner][S_.PRODUCTION_BUGS].append(item)
        else:
            if (item.taskType == S_.DEV_TIME_BUGS):            
                self.allocations[bucketToUse][owner][S_.DEV_TIME_BUGS].append(item)
            else:
                self.allocations[bucketToUse][owner][S_.UNFINISHED_STORIES].append(item)
    
    def unAssign(self, item, owner, preferredBucket):
        oldOwner = item.owner
        bucketToUse = preferredBucket
        if (oldOwner == None):
            oldOwner = S_.OWNER_TBD
        if (oldOwner == S_.OWNER_TBD):
            bucketToUse = S_.UNPLANNED
            
        x = self.allocations[bucketToUse][oldOwner][S_.PRODUCTION_BUGS]
        if item in x:
            x.remove(item)
        x = self.allocations[bucketToUse][oldOwner][S_.DEV_TIME_BUGS]
        if item in x:
            x.remove(item)        
        x = self.allocations[bucketToUse][oldOwner][S_.UNFINISHED_STORIES]
        if item in x:
            x.remove(item)

    def reAssign(self, item, owner, preferredBucket, originalBucket = None):
        if (originalBucket == None): # Means original bucket is same as destination bucket
            originalBucket = preferredBucket
        self.unAssign(item, owner, originalBucket)
        self.assign(item, owner, preferredBucket)
          

    # Returns the list of WorkItem objects that are owned by the given owner
    #
    # -owner: a string, corresponding to the person whose tasks we want to get
    # -bucket: is either CURRENT_SPRINT or NEXT_SPRINT
    def getWorkItems(self, owner, bucket):
        tasks = []
        tasks.extend(self.allocations[bucket][owner][S_.PRODUCTION_BUGS])
        tasks.extend(self.allocations[bucket][owner][S_.DEV_TIME_BUGS])
        tasks.extend(self.allocations[bucket][owner][S_.UNFINISHED_STORIES])
        
        return tasks

    # Helper method invoked inside the method committedTime
    def _populate(self, load, capacity):
        rejects_time = 0.0
        rejects_count = 0
        debug_time = 0.0
        debug_count = 0
        implementation_time = 0.0
        implementation_count = 0
        bandwidth = capacity
            
        for item in load[S_.PRODUCTION_BUGS]:
            time_delta = item.estimate * (1-item.percentAchieved)
            rejects_time += time_delta
            rejects_count += 1
            bandwidth -= time_delta
        for item in load[S_.DEV_TIME_BUGS]:
            time_delta = item.estimate * (1-item.percentAchieved)
            debug_time += time_delta
            debug_count += 1
            bandwidth -= time_delta
        for item in load[S_.UNFINISHED_STORIES]:
            time_delta = item.estimate * (1-item.percentAchieved)
            implementation_time += time_delta
            implementation_count += 1
            bandwidth -= time_delta
        return rejects_time, rejects_count, debug_time, debug_count, implementation_time, implementation_count, bandwidth
        
    
    # Returns a DataFrame with information of how much effort is already committed per developer to the
    # three different pairs buckets, each pair corresponding to the time (in man-days) and the count (number of
    # tickets or user stories) for these categories: 'Rejects' (PRODUCTION_BUGS), 'Debugging' (DEV_TIME_BUGS),
    # and 'Implementation' (UNFINISHED_STORIES). For each there is a '(days)' suffix for man-days and a '(#)' suffix for counts
    #
    # Result includes a 'Bandwidth' column indicating what capacity is left within the given capacity (in man-days)
    def committedTime(self, capacity):

        team = self.globalRepo.teamsRepo.findTeam(self.context.teamId)
        #cols = ['Developer', 'Rejects (days)', 'Rejects (#)', 'Debugging (days)', 'Debugging (#)', \
        #        'Implementation (days)', 'Implementation (#)', 'Bandwidth']
        result_dict = {'Developer': [], 'Rejects (days)': [], 'Rejects (#)': [], 'Debugging (days)': [], 'Debugging (#)': [], \
                       'Implementation (days)': [], 'Implementation (#)': [], 'Bandwidth': [], \
                       'NEXT SPRINT (days)': [], 'NEXT SPRINT (#)': [], 'NEXT SPRINT Bandwidth': []}
        owners = team.developers.copy()
        owners.append(S_.OWNER_TBD)
        for person in owners:
            result_dict['Developer'].append(person)

            load = {}
            if (person == S_.OWNER_TBD):
                load = self.allocations[S_.UNPLANNED][person]
            else:
                load = self.allocations[S_.CURRENT_SPRINT][person]

            r_d, r_c, d_d, d_c, i_d, i_c, bandwidth = self._populate(load, capacity)
        
            result_dict['Rejects (days)'].append(r_d)
            result_dict['Rejects (#)'].append(r_c)
            result_dict['Debugging (days)'].append(d_d)
            result_dict['Debugging (#)'].append(d_c)
            result_dict['Implementation (days)'].append(i_d)
            result_dict['Implementation (#)'].append(i_c)
            if (person != S_.OWNER_TBD):
                result_dict['Bandwidth'].append(bandwidth) # What is left from this sprint, not counting next sprint acceleration
            else:
                result_dict['Bandwidth'].append(None)
            
            # Now look at what is in store for the next sprint, if anything. We start off with a capacity for the
            # next sprint increased by any leftover bandwidth from the currrent sprint.
            if (person != S_.OWNER_TBD):
                load = self.allocations[S_.NEXT_SPRINT][person]
                r_d, r_c, d_d, d_c, i_d, i_c, bandwidth = self._populate(load, bandwidth + capacity)
                result_dict['NEXT SPRINT (days)'].append(r_d + d_d + i_d)
                result_dict['NEXT SPRINT (#)'].append(r_c + d_c + i_c) 
                result_dict['NEXT SPRINT Bandwidth'].append(bandwidth) 
            else: # just pad with 0's the data for the NEXT SPRINT
                result_dict['NEXT SPRINT (days)'].append(0)
                result_dict['NEXT SPRINT (#)'].append(0)        
                result_dict['NEXT SPRINT Bandwidth'].append(0) 

            
                
        return pd.DataFrame(result_dict)

    # Helper method used in the implementation of method 'committedTasks'
    def _committedTasks_helper(self, bucket, person, result_dict):
        load = self.allocations[bucket][person]
        
            
        for item in load[S_.PRODUCTION_BUGS]:
            ticket = self.globalRepo.ticketsRepo.findTicket(item.ticketId)

            result_dict['Owner'].append(person)
            result_dict['Task Type'].append(S_.PRODUCTION_BUGS)
            result_dict['Task Description'].append(item.ticketId)
            result_dict['User Story Id'].append(item.userStoryId)
            result_dict['Planned for Sprint'].append(item.sprintPlanned)
            result_dict['Most Recently Assigned in Sprint'].append(item.sprintPlanned)
            result_dict['Delivered in Sprint'].append(ticket.sprintFixed)
            result_dict['Original Estimate'].append(item.estimate)
            result_dict['Bucket'].append(bucket)
            result_dict['Effort Spent'].append(item.actual)
            result_dict['Effort Remaining'].append(item.estimate * (1-item.percentAchieved))
            result_dict['Percent Achieved'].append(item.percentAchieved)
            result_dict['Global Estimate'].append(ticket.estimatedCost)
            result_dict['% Global Done'].append(ticket.percentAchieved)
            
        for item in load[S_.DEV_TIME_BUGS]:
            ticket = self.globalRepo.ticketsRepo.findTicket(item.ticketId)
            
            result_dict['Owner'].append(person)
            result_dict['Task Type'].append(S_.DEV_TIME_BUGS)
            result_dict['Task Description'].append(item.ticketId)
            result_dict['User Story Id'].append(item.userStoryId)
            result_dict['Planned for Sprint'].append(item.sprintPlanned)
            result_dict['Most Recently Assigned in Sprint'].append(item.sprintPlanned)
            result_dict['Delivered in Sprint'].append(ticket.sprintFixed)
            result_dict['Original Estimate'].append(item.estimate)
            result_dict['Bucket'].append(bucket)
            result_dict['Effort Spent'].append(item.actual)
            result_dict['Effort Remaining'].append(item.estimate * (1-item.percentAchieved))
            result_dict['Percent Achieved'].append(item.percentAchieved)
            result_dict['Global Estimate'].append(ticket.estimatedCost)
            result_dict['% Global Done'].append(ticket.percentAchieved)
            
        for item in load[S_.UNFINISHED_STORIES]:
            uss   = self.globalRepo.teamsRepo.getUserStoryStatus(item.userStoryId)
            story = self.globalRepo.storiesRepo.findStory(item.userStoryId)

            result_dict['Owner'].append(person)
            result_dict['Task Type'].append(S_.UNFINISHED_STORIES)
            result_dict['Task Description'].append("Story implementation")
            result_dict['User Story Id'].append(item.userStoryId)
            result_dict['Planned for Sprint'].append(item.sprintPlanned)
            result_dict['Most Recently Assigned in Sprint'].append(uss.retrieveMostRecentAssignment())
            result_dict['Delivered in Sprint'].append(uss.sprintDelivered)
            result_dict['Original Estimate'].append(item.estimate)
            result_dict['Bucket'].append(bucket)
            result_dict['Effort Spent'].append(item.actual)
            result_dict['Effort Remaining'].append(item.estimate * (1-item.percentAchieved))
            result_dict['Percent Achieved'].append(item.percentAchieved)
            result_dict['Global Estimate'].append(story.originalEstimate)
            result_dict['% Global Done'].append(uss.percentAchieved)
    
    # Returns an informative dataframe of what user stories and tickets are allocated in this 'WorkAssignment' instance
    def committedTasks(self):       
        #cols = ['Owner', 'Task Type', 'User Story Id', 'Original Estimate', \
        #        'Effort Spent', 'Effort Remaining', 'Percent Achieved']
        result_dict = {'Owner': [], 'Task Type': [], 'Task Description': [],'User Story Id': [], 'Planned for Sprint': [], \
                       'Most Recently Assigned in Sprint': [], 'Delivered in Sprint': [], 'Original Estimate': [], \
                       'Bucket': [], 'Effort Spent': [], 'Effort Remaining': [], 'Percent Achieved': [], \
                      'Global Estimate': [], '% Global Done': []}
        team = self.globalRepo.teamsRepo.findTeam(self.context.teamId)
        for person in team.developers:
            self._committedTasks_helper(S_.CURRENT_SPRINT, person, result_dict)
            self._committedTasks_helper(S_.NEXT_SPRINT, person, result_dict)
            
        return pd.DataFrame(result_dict)

In [12]:
class UserStoryStatus:
# Represents what is pending to take the user story to the finishing line
#
# -userStoryId: a string unique identifier of the user story for which this is a status. Example: "UserStory #43"
# -percentAchieved: a number between 0.0 and 1.0, representing the percentage of the initial specification completed as determined
# by the responsible developer
# -planned: boolean to indicate if the user story has already been chosen to be worked in a sprint
# -sprintPlanned: first sprint in which the user story was chosen to be worked on. If not yet planned, has value S_.NOT_SET
# -sprintDelivered: first sprint during which the user story was fully achieved (i.e., self.percentAchieved=1.0). If not
# yet delivered, has value S_.NOT_SET
    def __init__(self, userStoryId, percentAchieved=0.0):
        self.userStoryId = userStoryId
        self.percentAchieved = percentAchieved
        self.planned = False # Determines if this user story was added to some sprint to do it. 
        self.sprintPlanned = S_.NOT_SET # Records in which sprint this user story was first planned to be delivered in
        self.sprintDelivered = S_.NOT_SET # Records in which sprint the story development was completed.
        
        # Internal set of pairs (integer, string) of (sprint, developer) to record each time a developer is assigned
        # work on this user story implementation. Pertinent to trace back work that has been re-assigned from one
        # developer to another
        self.assignmentEventsLog = [] 
     
    # Returns an array of WorkItem objects corresponding to all the work pending for this user story.
    # -storiesRepo: a UserStoriesRepo of all the user stories. Used to retrieve some information about the user story in 
    # question
    # -ticketsRepo: a TicketsRepo of all tickets against all user stories. Used to find pending tickets against
    # the user story in question.
    def _generateWorkItems(self, context, globalRepo):
        items = []
        story = globalRepo.storiesRepo.findStory(self.userStoryId)
        owner = S_.OWNER_TBD
        if self.planned:
            owner = story.developer
        # One item is the work in the user story itself, if it is not already implemented
        if self.percentAchieved < 1.0:
            item = WorkItem(self.userStoryId, S_.UNFINISHED_STORIES, story.originalEstimate*(1-self.percentAchieved), \
                            0.0, owner, self.sprintPlanned)
            items.append(item)
        # Now add an item for each pending ticket
        #
        # NOTE ON BANDWIDTH GOING NEGATIVE: Since tickets just came in, when this load is added on top of the prior load on a developer
        # (for example, prior load from stories started but not finished yet) the result may be that the developer
        # might become overbooked (i.e., more is allocated to the developer than they can do in the sprint)/
        # That is considered "normal"
        #
        # NOTE ON OWNER: we assign fixing the bug to the person who owns the user story
        openTickets = globalRepo.ticketsRepo.getOpenTickets(self.userStoryId)        
        for ticket in openTickets:
            
            # We should only have tickets reported in earlier sprints when generating work for this sprint
            assert ticket.sprintReported < context.sprint, 'TicketId='+ticket.ticketId + '; sprint='+str(context.sprint)
            
            sprintInWhichToFix = ticket.sprintReported + 1 # By default, it should get fixed in the next sprint after reporting it
            item = WorkItem(self.userStoryId, S_.DEV_TIME_BUGS, ticket.estimateRemainingEffort(), \
                            0.0, owner, sprintInWhichToFix, ticket.ticketId)
            items.append(item)
            
        # TODO: bugs on stories delivered, i.e., S_.PRODUCTION_BUGS

        return items
    
    
    # Reduces the amount of work left based on the completed work during this sprint, but then increases
    # the pending work by adding the newTickets that have come in
    #
    # -completedWork: a list of WorkItems, corresponding to the work items that were labored on during this sprint and 
    # whose 'percentAchieved' field represents how much of the work item we managed to complete in this sprint.
    # -newTickets: a list of Tickets, representing newly issued tickets that increment our backlog
    # -ctx: a ReleaseCycleContext instane
    def updateStatus(self, completedWork, ctx, globalRepo):
        for item in completedWork:
            if (item.userStoryId != self.userStoryId):
                continue # Do nothing, as the completedWork is not for this UserStoryStatus
            if item.taskType==S_.UNFINISHED_STORIES:
                # We developed the user story with an estimate that corresponds to a fraction of (1-percentAchieved) of the
                # user story. So if we completed only 70% of the estimate, say, then we completed 0.7*(1-percentAchieved), so we
                # still have 0.3 * (1 - percentAchieved) to finish the story. So the new value of percentAchieved is as per this
                # formula
                old_percentAchieved = self.percentAchieved
                self.percentAchieved = 1 - (1 - item.percentAchieved)*(1-self.percentAchieved)
                
                # To fix a bug with rounding (manifesting itself as still trying to work on user stories that
                # have been finished), treat it 100% achieved if we are close enough
                EPSILON = 0.00001
                if abs(self.percentAchieved - 1.0) < EPSILON:
                    self.percentAchieved = 1.0
                
                # If the algorithms to simulate each sprint work properly, we shouldn't have a work item to develop
                # the story further once it is fully finished, so assert
                assert(1.0 > old_percentAchieved)
                # Thanks to prior assertion, we know that self.percentAchieved will reach a value of 1.0 exactly once,
                # and that is for the sprint in which it is completed. This justifies setting the sprintDelivered
                # attribute and leave a record of when we completed this story.
                if self.percentAchieved == 1.0:
                    self.sprintDelivered = ctx.sprint
            else: 
                # We are working on a ticket then.
                ticket = globalRepo.ticketsRepo.findTicket(item.ticketId)
                ticket.effortToDate = ticket.effortToDate + item.actual
                
                assert(item.percentAchieved <= 1.0) #percentAchieved should never be more than 100% done
                
                assert(ticket.sprintReported < ctx.sprint) # We can only fix ticket in sprint(s) after ticket is reported
                
                # This formula works both when we didn't finish the ticket as when we did, i.e.,
                # if item.percentAchieved=1 then it will result in ticket.percentAchieved=1
                ticket.percentAchieved = 1 - (1 - item.percentAchieved)*(1-ticket.percentAchieved)
                
                # To fix a bug with rounding (manifesting itself as still trying to work on tickets that
                # have been finished), treat it 100% achieved if we are close enough
                EPSILON = 0.00001
                if abs(ticket.percentAchieved - 1.0) < EPSILON:
                    ticket.percentAchieved = 1.0
                
                if ticket.percentAchieved == 1.0: # Record this is the sprint we fixed the bug
                    ticket.sprintFixed = ctx.sprint
                    ticket.fixedBy     = item.owner
                
                globalRepo.ticketsRepo.updateTicket(ticket)
                
    # Record assignment event in uss
    def logAssignmentEvent(self, sprint, owner):
        self.assignmentEventsLog.append([sprint, owner])
        
    # Returns the sprint when this story was most recently assigned to a developer
    def retrieveMostRecentAssignment(self):
        mostRecentAssignment = S_.NOT_SET
        if len(self.assignmentEventsLog) > 0:
            mostRecentAssignment = self.assignmentEventsLog[-1][0]
        return mostRecentAssignment


In [26]:
class Backlog:
# Represents the remaining work to be done by a scrum team to complete their tasks for a release.
#
# -TeamId: string identifiying the scrum team for which this is a backlog. For example: "Scrum Team B"
# -pendingUserStories: array of UserStoryStatus objects. It is the universe of all that a team has yet to do. Initially
# it is an empty array but as a GA Cycle is launced, the team should get assigned all user stories pertinent to their
# area and the initial UserStoryStatus would probably be 0% done, 0 tickets, and then evolve from that from one sprint 
# to the other.

    def __init__(self):
        self.pendingUserStories = []
    
    # Returns an array of WorkItem objects corresponding to all the work implied by this backlog
    # -repo: a UserStoriesRepo of all the user stories. Used to retrieve some information about the user story in 
    # question
    def _generateWorkItems(self, context, globalRepo):
        items = []
        for uss in self.pendingUserStories:
            items.extend(uss._generateWorkItems(context, globalRepo))
        return items
    
    # Reduces the amount of work left based on the completed work during this sprint, but then increases
    # the pending work by adding the newTickets that have come in
    #
    # -completedWork: a list of WorkItems, corresponding to the work items that were labored on during this sprint and 
    # whose 'percentAchieved' field represents how much of the work item we managed to complete in this sprint.
    # -newTickets: a list of Tickets, representing newly issued tickets that increment our backlog
    # -context: a ReleaseCycleContext instance
    def updateStatus(self, completedWork, context, globalRepo):
        for uss in self.pendingUserStories:
            uss.updateStatus(completedWork, context, globalRepo)
            
    # Looks into self.pendingUserStories and returns the SUserstoryStatus instance that matches the given storyId, or
    # None if there is no match
    def getUserStoryStatus(self, storyId):
        for uss in self.pendingUserStories:
            if uss.userStoryId == storyId:
                return uss
        return None

In [6]:
# Represents a technical member of the R&D team
class Developer:
    
    # -name: a string, representing the unique identifier of this developer. Example: 'Suzie Jones'
    # -profile: a string indicating the profile of the developer. Take from the static choices availalbe in 
    # simulation.statics
    def __init__(self, name, profile, salary, functional_area):
        self.name            = name
        self.profile         = profile
        self.salary          = salary
        self.functional_area = functional_area

In [7]:
# Represents an in-memory proxy for a repo of Developer instances
class DevelopersRepo:
    
    def __init__(self, developers):
        self.developers = developers
        
    # -name: string uniquely identifying the Developer instance being retrieved
    def findDeveloper(self, name):
        for developer in self.developers:
            if developer.name == name:
                return developer
        return None

In [27]:
class ScrumTeam:
# Represents the composition and duties of a scrum team
#
# -TeamId: string identifiying this scrum team. For example: "Scrum Team B"
# -developers: array of strings. Represents the developers (including QA) in this team. Each array element is the 
# name of a developer.
# -productManagers: array of strings. Represents the productManager(s) responsible for the specs that this scrumTeam must
# implement.
# -areasOfResonsibility: array of strings. Represents the functional areas that this scrum team has responsibility for.
# For example: '[Doctor, Patient]' for a Theia scrum team that is responsible for both the 'Doctor' and 'Patient' bounded
#contexts.
# -backLog: a BackLog object. Represents all the remaning work that this scrum team has to do for the release.

    def __init__(self, teamId, developers=[], productManagers=[], areasOfResponsibility=[]):
        self.teamId = teamId
        self.developers = developers
        self.productManagers = productManagers
        self.areasOfResponsibility = areasOfResponsibility
        self.backlog = Backlog()

In [28]:
# Represents an in-memory proxy for a repo of ScrumTeam instances.
class ScrumTeamsRepo:

    # -teams: a list of ScrumTeam objects.
    def __init__(self, teams):
        self.teams = teams
        
    def findTeam(self, teamId):
        for team in self.teams:
            if team.teamId == teamId:
                return team
        return None
    
    def getUserStoryStatus(self, storyId):
        for team in self.teams:
            for uss in team.backlog.pendingUserStories:
                if uss.userStoryId == storyId:
                    return uss
        return None
    
    def getTeamId(self, storyId):
        for team in self.teams:
            for uss in team.backlog.pendingUserStories:
                if uss.userStoryId == storyId:
                    return team.teamId
        return None

In [ ]:
# Represents an aggregated in-memory proxy for all repos pertinent to the simulation of a release process
class GlobalRepo:
    def __init__(self, developersRepo, teamsRepo, storiesRepo, ticketsRepo):
        self.teamsRepo       = teamsRepo
        self.developersRepo  = developersRepo
        self.storiesRepo     = storiesRepo
        self.ticketsRepo     = ticketsRepo


<h1>Non-Stochastic Data Generators</h1>

In [5]:
def createTeamsDF(developers_df, productManagers_df):
# Create the teams, as a DataFrame.
# Each team is a ScrumTeam object, but we also unwrap (i.e., duplicate) a team's attributes into columns 
# of the dataframe created so that by viewing the dataframe we can verify that ScrumTeam objects were created correctly. 
# The 'Scrum Team' column in the dataframe is the actual ScrumTeam object, containing in object form what the other 
# dataframe columns display
#
# -developers_df: dataFrame of the developer information, each row having a developers name, scrum team label, and areas
# that developer works on (i.e., which Functional Areas)
# -productManagers_df: dataframe of the productManagers information, each row having the PM name and the areas for which
# that PM writes specs.
#
    teams_dict = {'Team Id': [], 'Developers': [], 'Product Managers': [], 'Areas of Responsibility': [], 'Scrum Team': []}
    teamLabels = developers_df['Scrum Team'].unique()
    for l in teamLabels:
        teamId = 'Team ' + l
        developers = list(developers_df[developers_df['Scrum Team'] == l]['Name'].apply(lambda x: x.rstrip('\xa0').strip()))
        areasOfResponsibility = list(developers_df[developers_df['Scrum Team'] == l]['Functional Area'].unique())
        productManagers = []
        for area in areasOfResponsibility:
            productManagers.extend(list(productManagers_df[productManagers_df['Functional Area']==area]['PM'].apply(lambda x: x.rstrip('\xa0').strip())))
        team = ScrumTeam(teamId, developers, productManagers, areasOfResponsibility)
        teams_dict['Team Id'].append(team.teamId)
        teams_dict['Developers'].append(team.developers)
        teams_dict['Product Managers'].append(team.productManagers)
        teams_dict['Areas of Responsibility'].append(team.areasOfResponsibility)    
        teams_dict['Scrum Team'].append(team) # this duplicates the other entries, but is to have information also packaged as an object
    teams_df = pd.DataFrame(teams_dict)
    return teams_df